In [1]:
import numpy as np
import pandas as pd

In this notebook we merge the three different datasets so we have one single table to work with.

In [2]:
def reduce_memory_usage(data, verbose = True):
    df = data.copy()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print(f"Memory usage decreased to {np.around(end_mem, 2)} Mb({np.around(100 * (start_mem - end_mem) / start_mem, 2)}% decrease)")    
    return df

In [3]:
%%time
sales = pd.read_csv("../data/raw/sales_train_validation.csv")

CPU times: user 4.64 s, sys: 633 ms, total: 5.27 s
Wall time: 5.32 s


In [4]:
%%time
calendar = pd.read_csv("../data/raw/calendar.csv")

CPU times: user 5.76 ms, sys: 1.62 ms, total: 7.38 ms
Wall time: 6.5 ms


In [5]:
%%time
prices = pd.read_csv("../data/raw/sell_prices.csv")

CPU times: user 1.93 s, sys: 246 ms, total: 2.17 s
Wall time: 2.19 s


Before merging datasets we change the column types to reduce memory usage.

In [6]:
reduced_sales = reduce_memory_usage(sales)

Memory usage decreased to 95.0 Mb(78.72% decrease)


In [7]:
reduced_calendar = reduce_memory_usage(calendar)

Memory usage decreased to 0.12 Mb(41.94% decrease)


In [8]:
reduced_prices = reduce_memory_usage(prices)

Memory usage decreased to 130.48 Mb(37.5% decrease)


Next we melt the sales data such that the days columns are merged into one column and the values in that column indicates the day.

In [9]:
melt_df = pd.melt(reduced_sales, 
                  id_vars = ['id','item_id','dept_id','cat_id','store_id','state_id'], 
                  var_name = "d", 
                  value_name = "sales")

In [10]:
melt_df.shape

(58327370, 8)

Now we can join the sales data with the calendar data and the price data.

In [11]:
merge_df = melt_df.merge(reduced_calendar, on = "d", how = "left")

In [12]:
merge_df.shape

(58327370, 21)

In [13]:
merge_df = merge_df.merge(reduced_prices, on = ["store_id", "item_id", "wm_yr_wk"], how = "left")

In [14]:
merge_df.shape

(58327370, 22)

Before saving the merged table we change the columns to their correct data types.

In [15]:
merge_df["date"] = pd.to_datetime(merge_df["date"])

In [16]:
merge_df.dtypes

id                      object
item_id                 object
dept_id                 object
cat_id                  object
store_id                object
state_id                object
d                       object
sales                    int16
date            datetime64[ns]
wm_yr_wk                 int16
weekday                 object
wday                      int8
month                     int8
year                     int16
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
snap_CA                   int8
snap_TX                   int8
snap_WI                   int8
sell_price             float16
dtype: object

In [17]:
%%time
merge_df.to_feather("../data/interim/merged_raw_data.ftr")

CPU times: user 27.6 s, sys: 32.9 s, total: 1min
Wall time: 1min 25s


In [18]:
%%time
df = pd.read_feather("../data/interim/merged_raw_data.ftr")

CPU times: user 22.1 s, sys: 12.4 s, total: 34.5 s
Wall time: 1min 30s


In [19]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
